In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/Hackthons/gpt3-instruct-sandbox"

/content/drive/MyDrive/Hackthons/gpt3-instruct-sandbox


In [ ]:
!pip install openai
!pip install streamlit
!npm install -g localtunnel

In [ ]:
import os
import sys
import pandas as pd
from sqlalchemy import create_engine
import ipywidgets as widgets
from IPython.display import display
from IPython.display import update_display
from IPython.display import display_pretty

from api import GPT, Example

In [ ]:
#Define display flow for GPT question input
def on_button_clicked(b):
    def sql_button_clicked(b):
        df = pd.read_sql(query, engine)
        df_pretty = df.style.hide_index().set_properties(**{'background-color': 'black',
                               'color' : 'lawngreen',
                               'border-color': 'white'})
        display(df_pretty)
        
    print ('\033[1mInput:\033[0m ' + inp.value)
    output = gpt.submit_request(inp.value)
    result = output['choices'][0].text
    query = result.split('output:')[1]
    print ('\033[1mGPT-3 Response:\033[0m ' + query)
    button2 = widgets.Button(description="Run SQL")
    button2.on_click(sql_button_clicked)
    display(button2)

In [ ]:
#Construct GPT-3-instruct instance, add instruction and examples
gpt = GPT(engine="davinci-instruct-beta",
          temperature=0.3,
          max_tokens=200)
gpt.add_instruction('Given an input question, respond with syntactically correct PostgreSQL.')

gpt.add_example(Example('select columns from users table', 
                        'select id, email, dt, plan_type, created_on, updated_on from users'))
gpt.add_example(Example('select columns from the charges table', 
                        'select amount, amount_refunded, created, customer_id, status from charges'))
gpt.add_example(Example('select columns from the customers table', 
                        'select created, email, id from customers'))
gpt.add_example(Example('how many users signed up in the past 30 days?', 
                        "SELECT COUNT(*) FROM users WHERE signup_dt >= now() - interval '30 days'"))
gpt.add_example(Example('when did user with email brian@seekwell.io sign up?', 
                        "SELECT signup_dt FROM users WHERE email = 'brian@seekwell.io'"))
gpt.add_example(Example('how much revenue did we have in the past 7 days?', 
                        "SELECT SUM(amount) from charges WHERE charge_dt >= now() - interval '7 days'"))
gpt.add_example(Example('how many users signed up in the past 30 days?', 
                        "SELECT COUNT(*) FROM users WHERE signup_dt >= now() - interval '30 days'"))
gpt.add_example(Example('how much revenue did we have from 10-01-20 through 11-15-20?', 
                        "SELECT SUM(case when charge_dt>= '10-01-20'::date and charge_dt < '11-15-20'::date then amount else 0 end) as revenue FROM charges"))
gpt.add_example(Example('how much revenue have we had from users that signed up in the last 6 months?', 
                        "SELECT SUM(charges.amount) FROM users INNER JOIN charges ON users.id = charges.user_id WHERE users.signup_dt>= now() - interval '6 months'"))
gpt.add_example(Example('when did user with email brian@seekwell.io make his first payment?', 
                        "SELECT MIN(charge_dt) as last_payment_dt from users INNER JOIN charges ON users.id = charges.user_id WHERE users.email = 'brian@seekwell.io'"))
gpt.add_example(Example('how many new users signed up in each of the last 2 months?', 
                        "SELECT sum(case when signup_dt>= now() - interval '1 month' then 1 else 0 end) as signups_this_month, sum(case when signup_dt>= now() - interval '2 months' and signup_dt < now() - interval '1 month'  then 1 else 0 end) as signups_last_month FROM users"))

In [ ]:
#Display UI to give GPT-3 prompt and run and display resulting SQL
inp = widgets.Text(description='Ask GPT-3:') # input field with label
button1 = widgets.Button(description="Get GPT-3 Response") # prediction button
#Box = widgets.HBox([inp, button1])
#print ('\033[1mInstruction:\033[0m ' + gpt.get_instruction_text())
#button1.on_click(on_button_clicked)
display(Box) # input field display

In [5]:
%%writefile requirements.txt
openai
requests
streamlit

Writing requirements.txt


In [4]:
%%writefile streamlit_app.py
# -*- coding: utf-8 -*-
"""
Created on Sunday June 26 2022
@author: Muhammad Ali
@github: @alihussainia
"""

import streamlit as st
from api import GPT, Example, set_openai_key
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

st.set_page_config(page_title="SQL Generator", layout='centered', initial_sidebar_state='auto', menu_items=None)

st.title("SQL Generator Application")
st.write("A web app that generates SQL queries using plain English")

inp = st.text_area("Please enter your query here", max_chars=2000, height=150)

# Get environment variables
key = os.getenv('key')
set_openai_key(key)

#Construct GPT-3-instruct instance, add instruction and examples
gpt = GPT(engine="davinci-instruct-beta",
          temperature=0.3,
          max_tokens=200)
gpt.add_instruction('Given an input question, respond with syntactically correct PostgreSQL.')

gpt.add_example(Example('select columns from users table', 
                        'select id, email, dt, plan_type, created_on, updated_on from users'))
gpt.add_example(Example('select columns from the charges table', 
                        'select amount, amount_refunded, created, customer_id, status from charges'))
gpt.add_example(Example('select columns from the customers table', 
                        'select created, email, id from customers'))
gpt.add_example(Example('how many users signed up in the past 30 days?', 
                        "SELECT COUNT(*) FROM users WHERE signup_dt >= now() - interval '30 days'"))
gpt.add_example(Example('when did user with email brian@seekwell.io sign up?', 
                        "SELECT signup_dt FROM users WHERE email = 'brian@seekwell.io'"))
gpt.add_example(Example('how much revenue did we have in the past 7 days?', 
                        "SELECT SUM(amount) from charges WHERE charge_dt >= now() - interval '7 days'"))
gpt.add_example(Example('how many users signed up in the past 30 days?', 
                        "SELECT COUNT(*) FROM users WHERE signup_dt >= now() - interval '30 days'"))
gpt.add_example(Example('how much revenue did we have from 10-01-20 through 11-15-20?', 
                        "SELECT SUM(case when charge_dt>= '10-01-20'::date and charge_dt < '11-15-20'::date then amount else 0 end) as revenue FROM charges"))
gpt.add_example(Example('how much revenue have we had from users that signed up in the last 6 months?', 
                        "SELECT SUM(charges.amount) FROM users INNER JOIN charges ON users.id = charges.user_id WHERE users.signup_dt>= now() - interval '6 months'"))
gpt.add_example(Example('when did user with email brian@seekwell.io make his first payment?', 
                        "SELECT MIN(charge_dt) as last_payment_dt from users INNER JOIN charges ON users.id = charges.user_id WHERE users.email = 'brian@seekwell.io'"))
gpt.add_example(Example('how many new users signed up in each of the last 2 months?', 
                        "SELECT sum(case when signup_dt>= now() - interval '1 month' then 1 else 0 end) as signups_this_month, sum(case when signup_dt>= now() - interval '2 months' and signup_dt < now() - interval '1 month'  then 1 else 0 end) as signups_last_month FROM users"))

submit_button = st.button('Generate')

if submit_button and inp=="":
  st.write("Please enter your problem above")

elif submit_button and inp!="":
  output = gpt.submit_request(inp.value)
  result = output['choices'][0].text
  query = result.split('output:')[1]
  st.markdown(query) 


st.text("App developed with ❤️ by @alihussainia")

st.text(f"Connect with me via Email at malirashid1994@gmail.com")

Writing streamlit_app.py


In [ ]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501

# Install Dependencies

In [ ]:
!apt install zstd

# the "slim" version contain only bf16 weights and no optimizer parameters, which minimizes bandwidth and memory
!time wget -c https://the-eye.eu/public/AI/GPT-J-6B/step_383500_slim.tar.zstd

!time tar -I zstd -xf step_383500_slim.tar.zstd

!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt

# jax 0.2.12 is required due to a regression with xmap in 0.2.13
!pip install mesh-transformer-jax/ jax==0.2.12 tensorflow==2.5.0

In [ ]:
!git clone https://github.com/kingoflolz/mesh-transformer-jax.git
!pip install -r mesh-transformer-jax/requirements.txt
!pip install optax==0.0.9 transformers dm-haiku einops
!pip install ray

In [ ]:
!pip install mesh_transformer

# Setup Model

In [ ]:
import os
import requests 
from jax.config import config

colab_tpu_addr = os.environ['COLAB_TPU_ADDR'].split(':')[0]
url = f'http://{colab_tpu_addr}:8475/requestversion/tpu_driver0.1_dev20210607'
requests.post(url)

# The following is required to use TPU Driver as JAX's backend.
config.FLAGS.jax_xla_backend = "tpu_driver"
config.FLAGS.jax_backend_target = "grpc://" + os.environ['COLAB_TPU_ADDR']

In [ ]:
import time

import jax
from jax.experimental import maps
import numpy as np
import optax
import transformers
import mesh_transformer
import mesh_transformer.sampling

from mesh_transformer.checkpoint import read_ckpt_lowmem
from mesh_transformer.sampling import nucleaus_sample
from mesh_transformer.transformer_shard import CausalTransformer

In [ ]:
params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

params = {
  "layers": 28,
  "d_model": 4096,
  "n_heads": 16,
  "n_vocab": 50400,
  "norm": "layernorm",
  "pe": "rotary",
  "pe_rotary_dims": 64,

  "seq": 2048,
  "cores_per_replica": 8,
  "per_replica_batch": 1,
}

per_replica_batch = params["per_replica_batch"]
cores_per_replica = params["cores_per_replica"]
seq = params["seq"]


params["sampler"] = nucleaus_sample

# here we "remove" the optimizer parameters from the model (as we don't need them for inference)
params["optimizer"] = optax.scale(0)

mesh_shape = (jax.device_count() // cores_per_replica, cores_per_replica)
devices = np.array(jax.devices()).reshape(mesh_shape)

maps.thread_resources.env = maps.ResourceEnv(maps.Mesh(devices, ('dp', 'mp')))

tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer = transformers.GPT2TokenizerFast.from_pretrained('gpt2')

Here we create the network and load the parameters from the downloaded files.

In [ ]:
total_batch = per_replica_batch * jax.device_count() // cores_per_replica

network = CausalTransformer(params)

network.state = read_ckpt_lowmem(network.state, "step_383500/", devices.shape[1])

network.state = network.move_xmap(network.state, np.zeros(cores_per_replica))

/usr/local/lib/python3.7/dist-packages/jax/experimental/maps.py:412: UserWarning: xmap is an experimental feature and probably has bugs!
  warn("xmap is an experimental feature and probably has bugs!")


key shape (8, 2)
in shape (1, 2048)
dp 1
mp 8
Total parameters: 6053381344
read from disk/gcs in 189.236s


# Run Model

Finally, we are ready to infer with the model! The first sample takes around a minute due to compilation, but after that it should only take about 10 seconds per sample.

Feel free to mess with the different sampling parameters (top_p and temp), as well as the length of the generations (gen_len, causes a recompile when changed).

You can also change other things like per_replica_batch in the previous cells to change how many generations are done in parallel. A larger batch has higher latency but higher throughput when measured in tokens generated/s. This is useful for doing things like best-of-n cherry picking.

*Tip for best results: Make sure your prompt does not have any trailing spaces, which tend to confuse the model due to the BPE tokenization used during training.*

In [ ]:
# allow text wrapping in generated output: https://stackoverflow.com/a/61401455
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def infer(context, top_p=0.9, temp=1.0, gen_len=512):
    tokens = tokenizer.encode(context)

    provided_ctx = len(tokens)
    pad_amount = seq - provided_ctx

    padded_tokens = np.pad(tokens, ((pad_amount, 0),)).astype(np.uint32)
    batched_tokens = np.array([padded_tokens] * total_batch)
    length = np.ones(total_batch, dtype=np.uint32) * len(tokens)

    start = time.time()
    output = network.generate(batched_tokens, length, gen_len, {"top_p": np.ones(total_batch) * top_p, "temp": np.ones(total_batch) * temp})

    samples = []
    decoded_tokens = output[1][0]

    for o in decoded_tokens[:, :, 0]:
      samples.append(f"\033[1m{context}\033[0m{tokenizer.decode(o)}")

    print(f"completion done in {time.time() - start:06}s")
    return samples

print(infer("EleutherAI is")[0])

completion done in 61.43248200416565s
EleutherAI is a bot that runs within chat, able to read text, list, and randomise text, but with a limited number of responses. It is able to parse text and generate a message based on an input string. You can use it in various ways, but it can be best used for converting a user's input into formatted text, especially when you want a certain amount of responses to be used up.

Originally started for a user-made site called Megagun, it is developed to be fun, and is not yet actually useful as a tool.

In both Irc and Skype you can take advantage of the fact that eThereAI is allowed to give customisable, and even random responses. It works well with people who do not always want an answer to their question, as it can also generate canned responses such as "Tell me more", "What would you like to know?", "Just put your question in the box." and many more. In any case, an improvement over most existing bots would be helpful for most of the community.

S

In [ ]:
#@title  { form-width: "300px" }
top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
temp = 1 #@param {type:"slider", min:0, max:1, step:0.1}

context = """In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."""

print(infer(top_p=top_p, temp=temp, gen_len=512, context=context)[0])

completion done in 13.55186653137207s
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.

This is no typical New World Camelopardalis, but one of three subspecies of the South American camelopardis, a camel-like, ungulate native to the Andes and Odriozé biomes of Bolivia, Peru, Ecuador and Colombia. A spectacular wild horse of the genus is protected under law, and there are fewer than 1,500 in the wild, which is far too few for the species to have gone extinct.

The Andean unicorn is an endangered species of the Pecari, or peccary, family. These wild horses have a thick neck, broad skull, long, lanky legs and a dainty tuft of hair on their backs and tails. “It looks really kind of cute,” as Donald Trump would say, but I never understood why, out of all the different varieties of native ungulates, they always 